<a href="https://colab.research.google.com/github/asliakalin/ML/blob/master/CNN_Text_Classification_wBackprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2: Convolutional Neural Networks for Text Classification

In this homework, you will be implementing the _forward pass_ and _backpropagation_ for a convolutional neural network with sparse inputs for text classification. 

## The setup
Let's define parameters for the Convolutional Neural Network. You do not need to modify them.

In [0]:
import sys
import math
import numpy as np
from collections import defaultdict

# window size for the CNN
width = 2

# number of filters
F = 100

# learning rate
alpha = 1e-1

# vocabsize: size of the total vocabulary
vocabsize = 10000

# vocab: the vocabulary dictionary with the word as key and its index as value
# the input will be transformed into respective positional indices using the vocab dictionary
# as the input for the forward and backward algorithm
# e.g. if vocab = {'a': 0, 'simple': 1, 'sentence': 2} and the training data is
# "a simple simple sentence a",
# the input to the forward and backward algorithm will be [0,1,1,2,0]
vocab = {}

np.random.seed(1)

# U and V are weight vectors of the hidden layer
# U: a matrix of weights of all inputs for the first
# hidden layer for all F filters in the
# where each filter has the size of vocabsize by width (window size)
# U[i, j, k] represents the weight of filter u_j for word with vocab[word] = i when the word is at the position k of the sliding window

# e.g. for the example, "a simple simple sentence a",
# if the window size is 4 and we are looking at the first sliding window
# of the 9th filter, the weight for the last "sentence" will be U[2, 8, 3]

# U[index of the word in vocab, index of the filter, position of the word in that sliding window]
U = np.random.normal(loc=0, scale=0.01, size=(vocabsize, F, width))

# V: the the weight vector of the F filter outputs (after max pooling)
# that will produce the output, i.e. o = sigmoid(V*h)
V = np.random.normal(loc=0, scale=0.01, size=(F))



In [0]:
w_i = np.zeros(vocabsize)
w_i[2] = 1

w_ii = np.zeros(vocabsize)
w_ii[3] = 1

val = np.tanh(U[2, 0, 0]+ U[3, 0, 1])
#print(U)     

Let's define some utility functions that may be useful. You don't need to modify them.

In [0]:
def sigmoid(x):
    """
    helper function that computes the sigmoid function
    """
    return 1. / (1 + math.exp(-x))


def read_vocab(filename):
    """
    helper function that builds up the vocab dictionary for input transformation
    """
    file = open(filename)
    for line in file:
        cols = line.rstrip().split("\t")
        word = cols[0]
        idd = int(cols[1])
        vocab[word] = idd
    file.close()


def read_data(filename):
    """
    :param filename: the name of the file
    :return: list of tuple ([word index list], label)
    as input for the forward and backward function
    """
    data = []
    file = open(filename)
    for line in file:
        cols = line.rstrip().split("\t")
        label = int(cols[0])
        words = cols[1].split(" ")
        w_int = []
        for w in words:
            # skip the unknown words
            if w in vocab:
                w_int.append(vocab[w])
        data.append((w_int, label))
    file.close()
    return data


def train():
    """
    main caller function that reads in the names of the files
    and train the CNN to classify movie reviews
    """
    vocabFile = "vocab.txt"
    trainingFile = "movie_reviews.train"
    testFile = "movie_reviews.dev"

    read_vocab(vocabFile)
    training_data = read_data(trainingFile)
    test_data = read_data(testFile)

    for i in range(50):
        #print(i)
        # confusion matrix showing the accuracy of the algorithm
        confusion_training = np.zeros((2, 2))
        confusion_validation = np.zeros((2, 2))

        for (data, label) in training_data:
            # back propagation to update weights for both U and V
            backward(data, label)

            # calculate forward and evaluate
            prob = forward(data)["prob"]
            pred = 1 if prob > .5 else 0
            confusion_training[pred, label] += 1

        for (data, label) in test_data:
            # calculate forward and evaluate
            prob = forward(data)["prob"]
            pred = 1 if prob > .5 else 0
            confusion_validation[pred, label] += 1

        print("Epoch: {} \tDev accuracy: {:.3f}"
            .format(
            i,
            np.sum(np.diag(confusion_validation)) / np.sum(confusion_validation)))

And finally, we'll download the data. We'll be doing sentiment analysis on a dataset of movie reviews, so we'll need 3 files - a vocabulary file, a file with a training set of movie reviews, and a development set containing different reviews.

In [0]:
%%capture
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_2/vocab.txt 
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_2/movie_reviews.dev
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_2/movie_reviews.train

## 1. Forward

Given the parameters and definition of the CNN model (§2 of HW), complete the Forward Function to calculate _o_ (the probability of the positive class) for an input text. You may not import any additional libraries. 

In [0]:
w_i = np.zeros(5)
w_i[1] = 1

w_ii = np.zeros(5)
w_ii[3] = 1

x_i = np.stack((w_i, w_ii), axis=1)
print(x_i)

[[0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]]


In [0]:
def forward(word_indices):
    """
    :param word_indices: a list of word indices, i.e. idx = vocab[word]
    :return: a result dictionary containing 3 items -
    result['prob']: output of the CNN algorithm.
    result['h']: the hidden layer output after max pooling, h = [h1, ..., hF]
    result['hid']: argmax of F filters, e.g. j of x_j
    e.g. for the ith filter u_i, tanh(word[hid[i], hid[i] + width]*u_i) = max(h_i)
    """

    h = np.zeros(F, dtype=float)
    hid = np.zeros(F, dtype=int)
    prob = 0.0

    # step 1. compute h and hid
    # loop through the input data of word indices and
    # keep track of the max filtered value h_i over all windows using filter i, and its position index x_j
    # h_i = max(tanh(weighted sum of all words in a given window over all windows for u_i
    
    for j in range(F): # j represents filter
      p_j = np.zeros(len(word_indices)-width+1) 
      # array of vals using same filter across different windows
      # same size as number of window sequences

      for i in range(len(word_indices)-width+1): # i represents the window starting position
        p_j[i] = np.tanh(U[word_indices[i], j,0] + U[word_indices[i+1], j, 1]) 
        # adds resulting value of i'th window sequence to index i of p_j

      h[j] = max(p_j)
      hid[j] = np.argmax(p_j) 

    # step 2. compute probability
    # once h and hid are computed, compute the probabiliy by sigmoid(h^TV)
    prob = sigmoid(np.dot(h, V))
    
    # step 3. return result
    return {"prob": prob, "h": h, "hid": hid}

## 2. Backward

Using the gradient update equations for V (§3 in HW) and U (§3.1), implement the updates for U and V in the backward function.

In [0]:
u_i = np.stack([U[5,7,0], U[10,7,1]])
u_i.shape

(2,)

In [0]:
def backward(word_indices, true_label):
    """
    :param word_indices: a list of word indices, i.e. idx = vocab[word]
    :param true_label: true label (0, 1) of the movie reviews
    :return: None
    update weight matrix/vector U and V based on the loss function
    """
    global U, V
    pred = forward(word_indices)
    prob = pred["prob"]
    h = pred["h"]
    hid = pred["hid"]

    # update U and V here
    # loss_function = y * log(o) + (1 - y) * log(1 - o) 
    #               = true_label * log(prob) + (1 - true_label) * log(1 - prob)
    # to update V: V_new = V_current + d(loss_function)/d(V)*alpha
    # to update U: U_new = U_current + d(loss_function)/d(U)*alpha
    # U[i, j, k] represents the weight of filter u_j for word with vocab[word] = i when the word is at the position k of the sliding window
    N = len(word_indices)
    window = width
    
    # L wrt h:
    dLdh = (true_label - prob)*V

    # h wrp p, L wrt p:
    dLdp = np.zeros((N-window+1, F))
    dhdp = np.zeros((F, N-window+1))
    for j in range(F):
      argmax_i = hid[j]
      dhdp[j][argmax_i] = 1
      dLdp[argmax_i][j] = 1*dLdh[j]

    #p wrt u
    dpdu = np.zeros((vocabsize, F, window))
    for j in range(F):
      argmax_i = hid[j]
      x_i = np.zeros((vocabsize, window))
      x_i[word_indices[argmax_i]][0] = 1
      x_i[word_indices[argmax_i+1]][1] = 1
      u_i = np.stack([U[word_indices[argmax_i],j,0], U[word_indices[argmax_i+1],j,1]])
      dpdu[word_indices[argmax_i],j] = np.matmul((1 - (np.tanh(np.matmul(x_i,u_i))**2)), x_i)
      
    # update U and V
    for i in range(len(word_indices)-window): 
      for f in range(F):
        U[word_indices[i],f,0] += alpha*(dLdp[i][f]*dpdu[word_indices[i],f,0])
        U[word_indices[i+1],f,1] += alpha*(dLdp[i+1][f]*dpdu[word_indices[i+1],f,1]) 

    V = V + alpha * (true_label - prob) * h

Once you have implemented both the forward and backward functions, your can test out your implementations by training the model. To do so, run the `train` function in the cell below. If your implementations are correct, you should see the accuracy improve as the model trains (You will be graded based on the correctness of the implementations, not on this accuracy).

In [0]:
train()

Epoch: 0 	Dev accuracy: 0.520
Epoch: 1 	Dev accuracy: 0.560
